<a href="https://colab.research.google.com/github/rickylukman/Pemrosesan-Bahasa-Alami/blob/main/eksplorasi_constituency_parser_Ricky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import library yang dibutuhkan

In [19]:
import nltk
from nltk.parse.generate import generate
from nltk.parse import ViterbiParser

## Bagian 1

Pendefinisian CFG

In [20]:
grammar_1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V N VP | N VP | V VP | V N N 
  NP -> "Saya" | "Kamu" | "Dia" | "Kawan" | "orang tua"
  N -> "jari" | "tulang" | "orang" | "tua" | "jalan"
  V -> "gigit" | "gigit jari" | "banting" | "banting tulang" | "melihat" | "membantu" | "menyebrang" | "bersama"
  """)

Pendefinisian sebuah contoh kalimat, perhatikan bahwa kalimat ini mengandung ambiguitas

In [21]:
sent_11 = 'Saya gigit jari melihat Kawan'.split()
sent_21 = 'Dia banting tulang membantu orang tua'.split()
sent_31 = 'Saya melihat orang menyebrang jalan bersama Dia'.split()

Parsing dengan parser Top Down Chart.

Perhatikan parse tree yang dihasilkan > 1

In [22]:
td_parser = nltk.parse.TopDownChartParser(grammar_1)


for tree in td_parser.parse(sent_11):
    print(tree)
print("--")
for tree in td_parser.parse(sent_21):
    print(tree)
print("--")
for tree in td_parser.parse(sent_31):
    print(tree)

(S (NP Saya) (VP (V gigit) (N jari) (VP (V melihat) (NP Kawan))))
(S
  (NP Saya)
  (VP (V gigit) (VP (N jari) (VP (V melihat) (NP Kawan)))))
--
(S
  (NP Dia)
  (VP (V banting) (N tulang) (VP (V membantu) (N orang) (N tua))))
(S
  (NP Dia)
  (VP
    (V banting)
    (VP (N tulang) (VP (V membantu) (N orang) (N tua)))))
--
(S
  (NP Saya)
  (VP
    (V melihat)
    (N orang)
    (VP (V menyebrang) (N jalan) (VP (V bersama) (NP Dia)))))
(S
  (NP Saya)
  (VP
    (V melihat)
    (N orang)
    (VP (V menyebrang) (VP (N jalan) (VP (V bersama) (NP Dia))))))
(S
  (NP Saya)
  (VP
    (V melihat)
    (VP
      (N orang)
      (VP (V menyebrang) (N jalan) (VP (V bersama) (NP Dia))))))
(S
  (NP Saya)
  (VP
    (V melihat)
    (VP
      (N orang)
      (VP (V menyebrang) (VP (N jalan) (VP (V bersama) (NP Dia)))))))


Parsing dengan parser Bottom Up Chart.

Perhatikan parse tree yang dihasilkan > 1

In [23]:
bu_parser = nltk.parse.BottomUpChartParser(grammar_1)


for tree in bu_parser.parse(sent_11):
    print(tree)
print("--")
for tree in bu_parser.parse(sent_21):
    print(tree)
print("--")
for tree in bu_parser.parse(sent_31):
    print(tree)

(S
  (NP Saya)
  (VP (V gigit) (VP (N jari) (VP (V melihat) (NP Kawan)))))
(S (NP Saya) (VP (V gigit) (N jari) (VP (V melihat) (NP Kawan))))
--
(S
  (NP Dia)
  (VP
    (V banting)
    (VP (N tulang) (VP (V membantu) (N orang) (N tua)))))
(S
  (NP Dia)
  (VP (V banting) (N tulang) (VP (V membantu) (N orang) (N tua))))
--
(S
  (NP Saya)
  (VP
    (V melihat)
    (VP
      (N orang)
      (VP (V menyebrang) (VP (N jalan) (VP (V bersama) (NP Dia)))))))
(S
  (NP Saya)
  (VP
    (V melihat)
    (VP
      (N orang)
      (VP (V menyebrang) (N jalan) (VP (V bersama) (NP Dia))))))
(S
  (NP Saya)
  (VP
    (V melihat)
    (N orang)
    (VP (V menyebrang) (VP (N jalan) (VP (V bersama) (NP Dia))))))
(S
  (NP Saya)
  (VP
    (V melihat)
    (N orang)
    (VP (V menyebrang) (N jalan) (VP (V bersama) (NP Dia)))))


Parsing dengan Shift Reduce parser

Perhatikan proses shift-reduce hingga dicapai simbol Start (S)

In [24]:
sr_parser = nltk.ShiftReduceParser(grammar_1, trace=2)

for tree in sr_parser.parse(sent_11):
    print(tree)

Parsing 'Saya gigit jari melihat Kawan'
    [ * Saya gigit jari melihat Kawan]
  S [ 'Saya' * gigit jari melihat Kawan]
  R [ NP * gigit jari melihat Kawan]
  S [ NP 'gigit' * jari melihat Kawan]
  R [ NP V * jari melihat Kawan]
  S [ NP V 'jari' * melihat Kawan]
  R [ NP V N * melihat Kawan]
  S [ NP V N 'melihat' * Kawan]
  R [ NP V N V * Kawan]
  S [ NP V N V 'Kawan' * ]
  R [ NP V N V NP * ]
  R [ NP V N VP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP Saya) (VP (V gigit) (N jari) (VP (V melihat) (NP Kawan))))


Cek apakah Grammar memenuhi syarat CNF.

In [25]:
 print(grammar_1.is_chomsky_normal_form())

False


Ubah CFG ke CNF

In [26]:
grammar_1 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V KT | N VP | V VP | V KP
  KT -> N VP
  KP -> N N
  NP -> "Saya" | "Kamu" | "Dia" | "Kawan" | "orang tua"
  N -> "jari" | "tulang" | "orang" | "tua" | "jalan"
  V -> "gigit" | "gigit jari" | "banting" | "banting tulang" | "melihat" | "membantu" | "menyebrang" | "bersama"
  """)

In [27]:
print(grammar_1.is_chomsky_normal_form())

True


## Bagian 2

Generate Grammar dari file constituency treebank

Menggunakan 5 kalimat awal dari Constituency Treebank Bahasa Indonesia, kethu https://github.com/ialfina/kethu

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
from nltk.corpus import BracketParseCorpusReader

ptb = BracketParseCorpusReader(r"/content/drive/My Drive/NLP_Data/DATA/", r".*/*\.mrg")

print(ptb)
print(ptb.sents())
print(ptb.parsed_sents())

<BracketParseCorpusReader in '/content/drive/My Drive/NLP_Data/DATA'>
[['Kera', 'untuk', '*', 'amankan', 'pesta', 'olahraga'], ['Pemerintah', 'kota', 'Delhi', 'mengerahkan', 'monyet', 'untuk', '*', 'mengusir', 'monyet-monyet', 'lain', 'yang', '*', 'berbadan', 'lebih', 'kecil', 'dari', 'arena', 'Pesta', 'Olahraga', 'Persemakmuran', '.'], ...]
[Tree('NP', [Tree('NN', ['Kera']), Tree('SBAR', [Tree('IN', ['untuk']), Tree('S', [Tree('NP-SBJ', [Tree('-NONE-', ['*'])]), Tree('VP', [Tree('VB', ['amankan']), Tree('NP', [Tree('NP', [Tree('NN', ['pesta']), Tree('NN', ['olahraga'])])])])])])]), Tree('S', [Tree('NP-SBJ', [Tree('NN', ['Pemerintah']), Tree('NN', ['kota']), Tree('NNP', ['Delhi'])]), Tree('VP', [Tree('VB', ['mengerahkan']), Tree('NP', [Tree('NN', ['monyet'])]), Tree('SBAR', [Tree('IN', ['untuk']), Tree('S', [Tree('NP-SBJ', [Tree('-NONE-', ['*'])]), Tree('VP', [Tree('VB', ['mengusir']), Tree('NP', [Tree('NP', [Tree('NN', ['monyet-monyet']), Tree('JJ', ['lain'])]), Tree('SBAR', [Tree('IN

Induksi PCFG (Probabilistic Context Free Grammar) dari constituency Treebank

In [30]:
from nltk import Nonterminal, nonterminals, Production, PCFG, induce_pcfg

S = Nonterminal('S')

productions = []
for t in ptb.parsed_sents():
    productions += t.productions()
grammar_3 = induce_pcfg(S, productions)
print(grammar_3)

Grammar with 6941 productions (start state = S)
    NP -> NN SBAR [0.019834]
    NN -> 'Kera' [0.000136593]
    SBAR -> IN S [0.557467]
    IN -> 'untuk' [0.06808]
    S -> NP-SBJ VP [0.343181]
    NP-SBJ -> -NONE- [0.385043]
    -NONE- -> '*' [0.366104]
    VP -> VB NP [0.225549]
    VB -> 'amankan' [0.000347947]
    NP -> NP [0.00728597]
    NP -> NN NN [0.0859138]
    NN -> 'pesta' [0.000136593]
    NN -> 'olahraga' [0.00081956]
    S -> NP-SBJ VP . [0.0796977]
    NP-SBJ -> NN NN NNP [0.000854701]
    NN -> 'Pemerintah' [0.00314165]
    NN -> 'kota' [0.000273187]
    NNP -> 'Delhi' [0.0012285]
    VP -> VB NP SBAR [0.0225264]
    VB -> 'mengerahkan' [0.000347947]
    NP -> NN [0.0581866]
    NN -> 'monyet' [0.00163912]
    VP -> VB NP PP [0.0872541]
    VB -> 'mengusir' [0.00139179]
    NP -> NP SBAR [0.0347096]
    NP -> NN JJ [0.0305606]
    NN -> 'monyet-monyet' [0.000273187]
    JJ -> 'lain' [0.0168539]
    IN -> 'yang' [0.159749]
    VP -> VB ADJP [0.00570288]
    VB -> 'berba

### Parse kalimat yang ada di constituency treebank

Parse sebuah kalimat dengan grammar hasil induksi menggunakan bottom-up parser

In [31]:
sent_12 = 'tetapi kemenangan Amerika menghasilkan keuntungan'.split()
# contoh menggunakan bottom-up parser
bu_parser = nltk.parse.BottomUpChartParser(grammar_3)

count = 0
for tree in bu_parser.parse(sent_12):
    print(tree)
    count += 1
    print("Pohon ke-",count+1)
    if (count == 9999): #Print 10 ribu data
        break

Streaming output truncated to the last 5000 lines.
          (NP (NNP Amerika) (NP (NP (FW (FW (VB menghasilkan))))))))))
  (VP (NN keuntungan)))
Pohon ke- 9491
(S
  (NP (ADJP (FW (FW (CC tetapi)))))
  (NP-SBJ-1
    (NP
      (NP
        (ADJP
          (NN kemenangan)
          (NP (NNP Amerika) (NP (FW (VB menghasilkan))))))))
  (VP (NN keuntungan)))
Pohon ke- 9492
(S
  (NP (ADJP (FW (FW (CC tetapi)))))
  (NP-SBJ-1
    (NP
      (NP
        (ADJP
          (NN kemenangan)
          (NP (NNP Amerika) (NP (FW (FW (VB menghasilkan)))))))))
  (VP (NN keuntungan)))
Pohon ke- 9493
(S
  (NP (ADJP (FW (FW (CC tetapi)))))
  (NP-SBJ-1
    (NP
      (NP
        (ADJP
          (NN kemenangan)
          (NP (NP (NNP Amerika)) (ADJP (FW (VB menghasilkan))))))))
  (VP (NN keuntungan)))
Pohon ke- 9494
(S
  (NP (ADJP (FW (FW (CC tetapi)))))
  (NP-SBJ-1
    (NP
      (NP
        (ADJP
          (NN kemenangan)
          (NP (NP (NNP Amerika)) (ADJP (FW (FW (VB menghasilkan)))))))))
  (VP (NN keuntung

Parsing dengan Viterbi Parser, yang akan mengembalikan 1 pohon parse dengan probability total paling tinggi

In [32]:
from nltk.parse import ViterbiParser

sent_12 = 'tetapi kemenangan Amerika menghasilkan keuntungan'.split()
parser = ViterbiParser(grammar_3, trace=2)
for t in parser.parse(sent_12):
    t.pretty_print()

Inserting tokens into the most likely constituents table...
   Insert: |=....| tetapi
   Insert: |.=...| kemenangan
   Insert: |..=..| Amerika
   Insert: |...=.| menghasilkan
   Insert: |....=| keuntungan
Finding the most likely constituents spanning 1 text elements...
   Insert: |=....| CC -> 'tetapi' [0.0112219]
   Insert: |=....| FW -> CC [0.00253807]
   Insert: |=....| CONJP -> CC [0.142857]
   Insert: |=....| NP -> FW [0.0107266]
   Insert: |=....| ADJP -> FW [0.0598086]
  Discard: |=....| FW -> FW [0.00507614]
   Insert: |=....| ADVP -> FW [0.00302115]
   Insert: |=....| NP-SBJ-1 -> FW [0.0025641]
  Discard: |=....| NP -> NP [0.00728597]
   Insert: |=....| NP-SBJ -> NP [0.00726496]
   Insert: |=....| FRAG -> ADJP [0.467213]
  Discard: |=....| FRAG -> NP [0.463115]
   Insert: |=....| NP-PRD -> NP [0.00568182]
  Discard: |=....| FRAG -> ADVP [0.0163934]
  Discard: |=....| FW -> FW [0.00507614]
   Insert: |=....| S -> NP-SBJ-1 [0.000343525]
   Insert: |=....| NP-SBJ-2 -> NP [0.00390

### Parse kalimat yang tidak ada di constituency treebank

Parse sebuah kalimat dengan grammar hasil induksi menggunakan bottom-up parser

In [33]:
sent_22 = 'kopi menjadi kebanggaan masyarakat kintamani'.split()
# contoh menggunakan bottom-up parser
bu_parser = nltk.parse.BottomUpChartParser(grammar_3)

for tree in bu_parser.parse(sent_22):
    print(tree)

ValueError: ignored

Parsing dengan Viterbi Parser, yang akan mengembalikan 1 pohon parse dengan probability total paling tinggi

In [34]:
sent_22 = 'kopi menjadi kebanggaan masyarakat kintamani'.split()
parser = ViterbiParser(grammar_3, trace=2)
for t in parser.parse(sent_22):
    t.pretty_print()

ValueError: ignored